In [5]:
# imports 
import math
import numpy as np
import scipy
from my_utils import black_scholes_price 
from my_utils import cf_BlackScholes

Price a 3-months European call option with the Black-Scholes
model. The initial stock price S0 equals 100, q = 0.01, r = 0.05
and σ = 0.25, K = 90, 95, 100, 105 and 110. Use the following
pricing methods

In [6]:
# variables 
S0 = 100
q = 0.01
r = 0.05
sigma = 0.25
K = [90, 95, 100, 105, 110]
T = 3/12

Price EC using Black Scholes

In [7]:
# price EC using Black Scholes
call_bs = []
for strike in K:
    call_price = black_scholes_price(S0, strike, T, r, q, sigma, 0)
    call_bs.append(round(call_price,4))

# Convert the list to a NumPy array
call_bs_array = np.array(call_bs)

print(f"Call Option Prices: {call_bs_array}")


Call Option Prices: [12.0002  8.3561  5.4584  3.3385  1.9129]


Price EC using FFT where the integral in the Carr-Madan formula is
approximated by the rectangular rule

In [8]:
# variables
eta = 0.25
N = 4096
alpha = 1.5
lambda_ = (2 * math.pi) / (N* eta)
b = (N * lambda_) / 2
print(lambda_)
print(b)

0.006135923151542565
12.566370614359172


In [9]:
# Define a finer grid from the original strikes to interpolate on (from min to max log strike)
log_strikes = np.arange(-b, b, lambda_)
print(log_strikes)

[-12.56637061 -12.56023469 -12.55409877 ...  12.54796284  12.55409877
  12.56023469]


In [10]:
# Grid for the Fourier transform variable
v = np.arange(0, N * eta, eta) 
print(v)
u = v - (alpha + 1) * 1j

[0.00000e+00 2.50000e-01 5.00000e-01 ... 1.02325e+03 1.02350e+03
 1.02375e+03]


In [11]:
# Define rho (g of x) using the characteristic function evaluated at (v - (alpha + 1) * i)
rho = np.exp(-r * T) * cf_BlackScholes(u, S0, r, q, sigma, T) / \
    (alpha**2 + alpha - v**2 + 1j * (2 * alpha + 1) * v)
print(rho)

[27804.87099528    +0.j         17024.27918315+21312.32603288j
 -5967.41147859+25116.12836105j ...    -0.            -0.j
    -0.            -0.j            -0.            -0.j        ]


In [12]:
# Apply the FFT using the rectangular rule
fft_result_rectangular  = np.fft.fft(np.exp(1j * v *b) * rho * eta, N)

In [13]:
# Simpson's Rule Coefficients
simpson_1 = 1/3  # First coefficient
simpson = (3 + (-1)**np.arange(2, N+1)) / 3  # Alternating coefficients starting from index 2
simpson_int = np.concatenate(([simpson_1], simpson))  # Combine with the first coefficient

# Apply Simpson's Rule correction in the FFT computation
fft_result_simpson = np.fft.fft(np.exp(1j * v * b) * rho * eta * simpson_int)

In [14]:
# Extract real part of the result
a_rectangular = np.real(fft_result_rectangular)
print(a_rectangular)
a_simpson = np.real(fft_result_simpson)

[3475.60887645 3475.60887647 3475.60887649 ... 3475.6088764  3475.60887641
 3475.60887643]


In [15]:
# Calculate call option prices using the Carr-Madan formula
calls_rectangular = (1 / np.pi) * np.exp(-alpha * log_strikes) * a_rectangular
calls_simpson = (1 / np.pi) * np.exp(-alpha * log_strikes) * a_simpson

In [16]:
from scipy.interpolate import interp1d

# Reconstruct strike prices from log-strikes
KK = np.exp(log_strikes)

# Compute call prices using FFT pricing formula for both rules
calls_rectangular = (1 / np.pi) * np.exp(-alpha * log_strikes) * a_rectangular
calls_simpson = (1 / np.pi) * np.exp(-alpha * log_strikes) * a_simpson

# Define specific strike prices for interpolation
specific_strikes = np.array([90, 95, 100, 105, 110])

# Create cubic spline interpolation functions
spline_rectangular = interp1d(KK, calls_rectangular, kind='cubic', fill_value="extrapolate")
spline_simpson = interp1d(KK, calls_simpson, kind='cubic', fill_value="extrapolate")

# Get interpolated option prices for specific strikes
interpolated_rectangular = spline_rectangular(specific_strikes)
interpolated_simpson = spline_simpson(specific_strikes)

# Print results
print(f"Interpolated Call Option Prices for K = {specific_strikes} using Rectangular Rule: {interpolated_rectangular}")
print(f"Interpolated Call Option Prices for K = {specific_strikes} using Simpson's Rule: {interpolated_simpson}")


Interpolated Call Option Prices for K = [ 90  95 100 105 110] using Rectangular Rule: [13.29598765  9.55088183  6.56474494  4.36676414  2.87179674]
Interpolated Call Option Prices for K = [ 90  95 100 105 110] using Simpson's Rule: [12.00024924  8.35608051  5.45842405  3.33851781  1.91285529]


In [17]:
# Print the call option prices
print("\nCall Option Prices using Black-Scholes:")
print(call_bs_array)

print("\nCall Option Prices using Rectangular Rule:")
print(interpolated_rectangular)

print("\nCall Option Prices using Simpson's Rule:")
print(interpolated_simpson)


Call Option Prices using Black-Scholes:
[12.0002  8.3561  5.4584  3.3385  1.9129]

Call Option Prices using Rectangular Rule:
[13.29598765  9.55088183  6.56474494  4.36676414  2.87179674]

Call Option Prices using Simpson's Rule:
[12.00024924  8.35608051  5.45842405  3.33851781  1.91285529]
